## **MoroccoAI Data Challenge (Edition 001)**

This notebook walks through The prcoccess of detecting plates from images using our 2 Fast-RCNN models that were trained on Plate Detection and Moroccan Plate Charachter Detection, and the post-processing that followed the predection.

<br>

### **Overview**

In Morocco, the number of registered vehicles doubled between 2000 and 2019. In 2019, a few months before lockdowns due to the Coronavirus Pandemic, 8 road fatalities were recorded per 10 000 registered vehicles. This rate is extremely high when compared with other IRTAD countries. The National Road Safety Agency (NARSA) established the road safety strategy 2017-26 with the main target to reduce the number of road deaths by 50% between 2015 and 2026 [1].Is crucial for law enforcement and authorities in order to assure the safety of the roads and to check the registration and the licence of the vehicles.
Therefore the aim to automate this task is very beneficial.

**This Jupyter Notebook only loads the trained Checkpoints, You can find Training Notebook in the next link.**

💡 Recommendation: [The Jupyter Notebook were we trained our models to detect Plates from pictures at first stage then detect Characters from Plates](https://colab.research.google.com/drive/1Niz1AVejRSm8UKFolP7DWla8WRpq6JwD?usp=sharing).


<br>

### **Dataset**

The dataset is 654 jpg pictures of the front or back of vehicles showing the license plate. They are of different sizes and are mostly cars. The plate license follows Moroccan standard.

For each plate corresponds a string (series of numbers and latin characters) labeled manually. The plate strings could contain a series of numbers and latin letters of different length. Because letters in Morocco license plate standard are Arabic letters, we will consider the following transliteration: a <=> أ, b <=> ب, j <=> ج (jamaa), d <=> د , h <=> ه , waw <=> و, w <=> w (newly licensed cars), p <=> ش (police), fx <=> ق س (auxiliary forces), far <=> ق م م (royal army forces), m <=>المغرب, m <=>M. For example:

the string “123ب45” have to be converted to “12345b”,<br>
the string “123و4567” to “1234567waw”,<br>
the string “12و4567” to “1234567waw”,<br>
the string “1234567ww” to “1234567ww”, (remain the same)<br>
the string “1234567far” to “1234567ق م م”,<br>
the string “1234567m” to “1234567المغرب", etc.
<br>

We offer the plate strings of 450 images (training set). The remaining 204 unlabeled images will be the test set. The participants are asked to provide the plate strings in the test set.

<br>

### **Our Approach & Models**

Our approach was to use Object Detection to detect plate characters from images. We have chosen to build two models separately instead of using libraries directly like easyOCR or Tesseract due to its weaknesses in handling the variance in the shapes of Moroccan License plates.
The first model was trained to detect the licence plate to be then cropped from the original image, which will be then passed into the second model that was trained to detect the characters. 

This notebook will be showing a code example on pretrained faster-rcnn model for both Object detection tasks,  using library called detectron2 developed by FaceBook AI Research Laboratory (FAIR) based on Pytorch.

<br>

### **Detectron2**

#### ![Detectron2 Logo](https://raw.githubusercontent.com/facebookresearch/detectron2/085fda47bc49f2cdd9c05a895580b2b31fcdb6c3/.github/Detectron2-Logo-Horz.svg)

Detectron2 is Facebook AI Research's next generation library that provides state-of-the-art detection and segmentation algorithms. It is the successor of Detectron and maskrcnn-benchmark. It supports a number of computer vision research projects and production applications in Facebook.


### **Fast-RCNN**

#### ![Fast-RCNN Architecture ](https://www.researchgate.net/profile/Akif-Durdu/publication/334987612/figure/fig3/AS:788766109224961@1565067903984/High-level-diagram-of-Faster-R-CNN-16-for-generic-object-detection-2-Inception-v2-The.ppm)

This version of the notebook doesn't contain the inference part since it's still on development. Once the inference and deployment part is done this will noteboook will be update.

### **About**

[MoroccoAI](https://morocco.ai/) MoroccoAI is an initiative led by AI experts in Morocco and abroad to promote AI growth in Morocco across the spectrum.


#### ![MoroccoAI Logo](https://morocco.ai/wp-content/uploads/2020/03/MoroccoAI_Logo.png)







<h2>Installing Detecron2<h2>
<p>
  First thing to do is try to Install detectron2 and Restart the runtime so all installed libraires get loaded.
</p>

In [31]:
!pip install pyyaml==5.1
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html

torch:  1.10 ; cuda:  cu111
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html


<h3>Importing libraries and Packages </h3>

In [32]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import statistics
import os, json, cv2, random
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
import pandas as pd
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import ColorMode

<h3>Importing Faster-RCNN model and loading checkpoint for licence Plates detection </h3>
Since this notebook is for running models to predict images from testset we will be loading our pretrained model and use "CPU" as MODEL.DEVICE

In [34]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.MODEL.DEVICE = "cpu"

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

#cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
#cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
#cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (licence). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)

In [35]:
cfg.MODEL.WEIGHTS = os.path.join("./Plate_Detection_Model", "model_final.pth")  # path to the model we just trained

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9   # set a custom testing threshold

predictor = DefaultPredictor(cfg)

<h3>Test folder</h3>
<p>in order to generate a csv file of plates countained on the folder,
you can pass the test folder's path  to Image_folder vriable
</p>

In [46]:
Images_folder = './test'

<h3>Extraction Folder</h3>
<p>This is the exporting folder for the plate image after extracting it from original images
</p>

In [47]:
Extraction_folder = './Plate_detection'

In [48]:
if not os.path.exists(Extraction_folder):
    os.makedirs(Extraction_folder)

<h3>Plates Extraction</h3>

<p>in order to handle images with multiple licence Plates.
We created two function one that extract all plates that exists in Image, and one that extract only the one plate that model predected with highest confidance
</p>

In [49]:
def Plates_Detection_All_Plates_In_Image(Images_folder,Image,Extraction_folder):
  im = cv2.imread(os.path.join(Images_folder, Image))
  outputs = predictor(im)  
  boxes = outputs['instances'].pred_boxes.tensor.cpu().numpy().tolist() 
  scores = outputs['instances'].scores.numpy().tolist()
  if len(scores)>0:
      classes = outputs['instances'].pred_classes.to('cpu').tolist()
      Plates = { i : boxes[i] for i in range(0, len(scores) ) }
      for k,v in Plates.items():
          cv2.imwrite(os.path.join(Extraction_folder, Image[:-4]+'_'+str(k)+'.jpg'), im[int(v[1]):int(v[3]), int(v[0]):int(v[2]), :])

In [50]:
def Plates_Detection_Top_Score_Plate_In_image(Images_folder,Image,Extraction_folder):
    im = cv2.imread(os.path.join(Images_folder, Image))
    outputs = predictor(im)  
    boxes = outputs['instances'].pred_boxes.tensor.cpu().numpy().tolist()
    scores = outputs['instances'].scores.numpy().tolist()
    if len(scores)>0:
        classes = outputs['instances'].pred_classes.to('cpu').tolist()
        Plates = { i : boxes[i] for i in range(0, len(scores) ) }
        Sorted_Plates_by_Score = sorted(Plates.items(), key=lambda e: e[1][0], reverse=True)
        Top_Score_Plate = Sorted_Plates_by_Score[0][1]
        cv2.imwrite(os.path.join(Extraction_folder, Image), im[int(Top_Score_Plate[1]):int(Top_Score_Plate[3]), int(Top_Score_Plate[0]):int(Top_Score_Plate[2]), :])

<p>if you want to take in considiration all licence  plates on each image,
the parameter all_plates should be set to True, otherwise keep the default variable False
 </p>

In [51]:
def Detect_Plates_From_Images(Images_folder,all_plates=False):
  for image in os.listdir(Images_folder):  
    if image.lower().endswith(('.png', '.jpg', '.jpeg')) :
      if all_plates :
        Plates_Detection_All_Plates_In_Image(Images_folder,image,Extraction_folder)
      else:
        Plates_Detection_Top_Score_Plate_In_image(Images_folder,image,Extraction_folder)

In the next Code we will be running our Main function which will be predicting and saving our detected plates into the extraction folder.

In [52]:
Detect_Plates_From_Images(Images_folder,all_plates=False)

/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride


<h3>Importing Faster-RCNN model and loading checkpoint for characters detector </h3>

In [53]:
cfg_ocr = get_cfg()
cfg_ocr.MODEL.DEVICE = "cpu"

cfg_ocr.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
#cfg_ocr.DATALOADER.NUM_WORKERS = 2
#cfg_ocr.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo

cfg_ocr.SOLVER.IMS_PER_BATCH = 2
cfg_ocr.SOLVER.STEPS = []        # do not decay learning rate
cfg_ocr.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # faster, and good enough for this toy dataset (default: 512)
cfg_ocr.MODEL.ROI_HEADS.NUM_CLASSES = 20 # Numbers of charachters that appears in moroccon licence plates

In [55]:
cfg_ocr.MODEL.WEIGHTS = os.path.join("./Characters_Detection_Model", "model_final.pth")  # path to the model we just trained

cfg_ocr.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold

predictor_ocr = DefaultPredictor(cfg_ocr)

We will create a dictionary that will take class ID (int) from predictor and return the exact string transformation like it was asked in the challenge 

In [57]:
classlist = ["0","1","2","3","4","5","6","7","8","9", "a","b","h","w","d","p","waw","j","m","m"]
classestolettres = { i : classlist[i] for i in range(0, len(classlist) ) }

<p>Function that return the characters and there bounding boxes(without order)</p>

In [58]:
def OCR_Predictor(Extraction_folder,Image):
    im = cv2.imread(os.path.join(Extraction_folder, Image))
    outputs = predictor_ocr(im)  
    boxes = outputs['instances'].pred_boxes.tensor.cpu().numpy().tolist()
    classes = outputs['instances'].pred_classes.to('cpu').tolist()
    dict_Of_predection = { i : [classes[i],boxes[i]] for i in range(0, len(outputs['instances'].pred_classes.to('cpu').tolist()) ) }
    return(dict_Of_predection)

This is post processing function aim to generate the right sequence of charachters to match the content of a licence plate

1.   Split characters based on median of Y_Min of all detected letters boxes, by taking characters  where their Y_Max is smaller than Median_Y_Mins into a string called top_characters, and those who have Y_Max greater than Median_Y_Mins will be in bottom_characters
2.   Order characters in top and bottom list from left to right based on the X_Min of the detected Box of each character




In [59]:
def OCR_Plates_Post_Processing(plate,dict_Of_predection):
    Plate = [item[1][1][1] for item in list(dict_Of_predection.items())]
    if len(Plate)<=0:
      ocr_result = {'plate_id':plate[:-4],'plate_string':''}
      return(ocr_result)
    medYmin = statistics.median(Plate)
    toplettres = dict()
    bottomlettres = dict()
    for k,v in dict_Of_predection.items():
      if (v[1][3] <= medYmin ):
        toplettres[k] = v
      else :
        bottomlettres[k] = v
    TopRes = sorted(toplettres.items(), key=lambda e: e[1][1][0])
    BottomRes = sorted(bottomlettres.items(), key=lambda e: e[1][1][0])
    TopPlate = [classestolettres[item[1][0]] for item in TopRes]
    BottomPlate = [classestolettres[item[1][0]] for item in BottomRes]
    TopPlate = "".join(str(x) for x in TopPlate)
    BottomPlate = "".join(str(x) for x in BottomPlate)
    ocr_result = {'plate_id':plate[:-4],'plate_string':BottomPlate+TopPlate}
    return(ocr_result)

this function is returning a pandas data frame of couples for image names and there prediction

In [60]:
def Get_OCR_From_Plates(Extraction_folder):
  column_names = ["plate_id", "plate_string"]
  submission_result = pd.DataFrame(columns = column_names)
  for plate in os.listdir(Extraction_folder):
    if plate.lower().endswith(('.png', '.jpg', '.jpeg')) :
      dict_Of_predection = OCR_Predictor(Extraction_folder,plate)
      ocr_result = OCR_Plates_Post_Processing(plate,dict_Of_predection)
      submission_result = submission_result.append(ocr_result, ignore_index=True)
      submission_result = submission_result.sort_values(by=['plate_id'], ascending=True)
  return(submission_result)

In [61]:
submission_result = Get_OCR_From_Plates(Extraction_folder)

/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride


In [62]:
submission_result

,plate_id,plate_string
0,638,88621a40
2,639,2905h6
1,640,37171waw6


<p>Saving the Data Frame as csv File</p>

In [ ]:
submission_result.to_csv("sample_submission.csv", encoding='utf-8', index=False)